# Circuit Optimization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import opics
from scipy import optimize
import scipy.signal as signal
from scipy import optimize

Import an OPICS component library

In [ ]:
ebeam = opics.libraries.ebeam

Define custom frequency data points for a component

In [ ]:
f = np.linspace(opics.C * 1e6 / 1.5, opics.C * 1e6 / 1.6, 1000)
lam = opics.C / f

Define a circuit to optimize

In [ ]:
def solve_circuit(tunable_params=[50 * 1e-6, 150 * 1e-6]):
    circuit_name = "mzi"
    circuit = opics.Network(network_id=circuit_name, f=f)
    # define component instances
    input_gc = circuit.add_component(ebeam.GC)
    y1 = circuit.add_component(ebeam.Y)
    wg1 = circuit.add_component(
        ebeam.Waveguide, params=dict(length=tunable_params[0] * 1e-6)
    )
    wg2 = circuit.add_component(
        ebeam.Waveguide, params=dict(length=tunable_params[1] * 1e-6)
    )
    y2 = circuit.add_component(ebeam.Y)
    output_gc = circuit.add_component(ebeam.GC)
    # define circuit connectivity
    circuit.connect(input_gc, 1, y1, 0)
    circuit.connect(y1, 1, wg1, 0)
    circuit.connect(y1, 2, wg2, 0)
    circuit.connect(y2, 0, output_gc, 1)
    circuit.connect(wg1, 1, y2, 1)
    circuit.connect(wg2, 1, y2, 2)

    return circuit.simulate_network().get_data(yscale="log")["S_1_0"]

In [ ]:
target_response = solve_circuit(tunable_params=[50, 100])
wg_lengths = [50, 150]
curr_response = solve_circuit(wg_lengths)
plt.plot(lam, target_response, "b")
plt.plot(lam, curr_response, "r")

Defined spectrum analyzing functions

In [ ]:
def get_peaks(x, y, plot=False):
    peaks, _ = signal.find_peaks(-y, prominence=1)
    if plot:
        plt.plot(x, y)
        plt.plot(x[peaks], y[peaks], "x")
        plt.show()

    if len(peaks) == 0:
        peaks = np.array([np.argmin(x), np.argmax(x)])
    elif len(peaks) == 1:
        peaks = np.repeat(peaks[0], 2)

    return x[peaks]


def get_closest_fsr_peaks(f, data, l0):
    peaks = get_peaks(f, data)

    fsr_pair = [(peaks[i - 1], peaks[i]) for i in range(1, len(peaks))]

    distances = []
    for each_fsr_pair in fsr_pair:
        if l0 >= each_fsr_pair[0] and l0 <= each_fsr_pair[1]:
            return np.array(each_fsr_pair)

        temp_val = 0
        temp_val += abs(each_fsr_pair[0] - l0)
        temp_val += abs(each_fsr_pair[1] - l0)
        distances.append(temp_val)
    idx_min = np.argmin(distances)
    return np.array(fsr_pair[idx_min])

In [ ]:
target_response = solve_circuit(tunable_params=[50, 100])
target_fsr_peaks = get_closest_fsr_peaks(lam * 1e9, target_response, 1550)
print(target_fsr_peaks)

Define a loss function, `loss`, and a forward-pass function, `fwd`, which outputs a scalar cost value to optimize.

In [ ]:
def loss(y_pred, y):
    val = np.sum(abs(y_pred - y))
    return val


def fwd(trainable_params, static_params):
    return loss(
        get_closest_fsr_peaks(lam * 1e9, solve_circuit(trainable_params), 1550),
        static_params,
    )

In [ ]:
wg_lengths = np.array([50, 150])
fwd(wg_lengths, target_fsr_peaks), get_closest_fsr_peaks(
    lam * 1e9, solve_circuit(wg_lengths), 1550
)

compute and plot the function in 3D

In [ ]:
all_a = np.arange(10, 200, 5)
all_b = np.arange(10, 200, 5)
z = np.zeros((all_a.size, all_b.size))
for ai in range(len(all_a)):
    for bi in range(len(all_b)):
        temp_grad = fwd([all_a[ai], all_b[bi]], target_fsr_peaks)
        z[ai, bi] = temp_grad

### Plot the objective function and the global minimum

In [ ]:
idx_min = np.where(z == np.min(z))
x_min = all_a[idx_min[0][0]]
y_min = all_b[idx_min[1][0]]
print(idx_min, x_min, y_min, np.amin(z))

In [ ]:
# Set up base figure: The contour map
from matplotlib import ticker

fig, ax = plt.subplots(figsize=(8, 6))
fig.set_tight_layout(True)
[A, B] = np.meshgrid(all_a, all_b)
img = ax.contourf(A, B, z.T, origin="lower", cmap="magma")
fig.colorbar(img, ax=ax)
ax.scatter(all_a[idx_min[0][:]], all_b[idx_min[1][:]], marker="x", color="red")

### zooming in 

In [ ]:
# Set up base figure: The contour map
fig, ax = plt.subplots(figsize=(8, 6))
fig.set_tight_layout(True)
slice = 30
[A, B] = np.meshgrid(all_a[:slice], all_b[:slice])
img = ax.contourf(A, B, z[:slice, :slice].T, origin="lower", cmap="magma")
ax.scatter(all_a[idx_min[0][:]], all_b[idx_min[1][:]], marker="x", color="red")
plt.xlim(10, 40)
plt.ylim(50, 80)
fig.colorbar(img, ax=ax)

In [ ]:
all_a[:7], all_b[:7]

### Hyper-parameters for the algorithm

### Functions

In [ ]:
import opics

from opics.optimization import Swarm


def fwd_vectorize(particle):
    _sum = 0.0
    _sum += fwd(particle.pos, target_fsr_peaks)
    return _sum


s = Swarm(20, 2, ((10.0, 200.0), (10.0, 200.0)), tolerance=-1)
s.optimize(fwd_vectorize, 1, 200)

In [ ]:
xx = ((-10.0, -20.0), (100.0, 200.0))
np.random.uniform(xx), np.array(
    [np.random.uniform(_[0], _[1]) for _ in xx]
), np.random.uniform(
    100.0,
    200.0,
    [
        2,
    ],
)

In [ ]:
(10.0, 200.0), np.amax((10.0, 200.0))

In [ ]:
np.random.uniform(0.39, 0.91, (2))

In [ ]:
target_response = solve_circuit(tunable_params=[50, 100])
wg_lengths = [50, 150]
curr_response = solve_circuit(s.gbestpos)
plt.plot(lam, target_response, "b")
plt.plot(lam, curr_response, "r")
plt.show()


print(get_peaks(lam * 1e9, solve_circuit(s.gbestpos), 1550), target_fsr_peaks)
print(
    loss(
        get_closest_fsr_peaks(lam * 1e9, solve_circuit(s.gbestpos), 1550),
        target_fsr_peaks,
    )
)

In [ ]:
s.log_best_particles[0][0]

plot animation

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
fig.set_tight_layout(True)
[A, B] = np.meshgrid(all_a, all_b)
img = ax.contourf(A, B, z.T, origin="lower", cmap="magma")
fig.colorbar(img, ax=ax)
ax.scatter(all_a[idx_min[0][:]], all_b[idx_min[1][:]], marker="x", color="green")
global arrow_ref
arrow_ref = []
for each_particle in s.log_all_particles[0]:
    p_arrow = ax.quiver(
        each_particle[0][0],
        each_particle[0][1],
        each_particle[1][0],
        each_particle[1][1],
        color="blue",
        width=0.005,
        angles="xy",
        scale_units="xy",
        scale=5,
    )
    arrow_ref.append(p_arrow)

gbest_plot = plt.scatter(
    s.log_best_particles[0][0],
    s.log_best_particles[0][1],
    marker="*",
    s=200,
    color="white",
    alpha=0.4,
)

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter


def animate(i):
    "Steps of PSO: algorithm update and show in plot"
    title = "Iteration {:02d}".format(i)
    ax.set_title(title)
    # Update params
    global arrow_ref
    for idx_particle in range(len(s.log_all_particles[i])):
        each_particle = s.log_all_particles[i][idx_particle]
        arrow_ref[idx_particle].set_offsets((each_particle[0][0], each_particle[0][1]))
        arrow_ref[idx_particle].set_UVC(each_particle[1][0], each_particle[1][1])
    gbest_plot.set_offsets(s.log_best_particles[i])


data_skip = 0


def init_func():
    ax.clear()


anim = FuncAnimation(
    fig,
    animate,
    frames=np.arange(0, len(s.log_all_particles) - 1, 1),
    interval=500,
    repeat=True,
)

In [ ]:
writergif = PillowWriter(fps=20)
anim.save("PSO_locations.gif", writer=writergif)

In [ ]:
xdata = np.arange(0, len(s.log_all_particles))
fig, ax = plt.subplots(figsize=(8, 6))
fig.set_tight_layout(True)
(lp,) = ax.plot(xdata, s.log_fitness)
plt.ylabel("Cost")
plt.xlabel("Iteration")

In [ ]:
def animate(i):
    "Steps of PSO: algorithm update and show in plot"
    title = "Iteration {:02d}".format(i)
    ax.set_title(title)
    # Update params
    lp.set_ydata(np.array(s.log_fitness)[:i])
    lp.set_xdata(xdata[:i])
    return (lp,)


data_skip = 0


def init_func():
    ax.clear()


anim = FuncAnimation(
    fig,
    animate,
    frames=np.arange(1, len(s.log_all_particles), 1),
    interval=500,
    repeat=True,
)

In [ ]:
writergif = PillowWriter(fps=20)
anim.save("PSO_cost.gif", writer=writergif)